In [1]:
import pandas as pd
import numpy as np
import math
import dask.array as da
import dask
from dask.distributed import LocalCluster, Client
import tifffile

In [2]:
def custom_filter(block):
    """
    Apply a custom filter to a 3D block.
    
    Parameters:
    - block: 3D numpy array, a small block of the image with padding.
    
    Returns:
    - filtered_block: 3D numpy array, the filtered block.
    """
    # Initialize the output block with zeros
    filtered_block = np.zeros(block.shape, dtype=block.dtype)
    
    # Iterate through each voxel in the block (excluding the border voxels due to padding)
    for z in range(1, block.shape[0] - 1):
        for y in range(1, block.shape[1] - 1):
            for x in range(1, block.shape[2] - 1):
                val = block[z, y, x]
                if val == 0:
                    neighbors = [block[z-1, y, x], block[z+1, y, x], block[z, y-1, x], block[z, y+1, x], block[z, y, x-1], block[z, y, x+1]]
                    array_1d = np.array(neighbors)
                    max_val = array_1d.max()
                    filtered_block[z, y, x] = max_val
                else:
                    filtered_block[z, y, x] = val
    
    # Return the filtered block, excluding the padding
    return filtered_block

def apply_custom_filter_dask(image, chunk_size=(100, 100, 100), iteration=1):
    """
    Apply a custom 3x3x3 filter to a 3D image using Dask.
    
    Parameters:
    - image: 3D numpy array, the input image.
    - chunk_size: tuple, the size of the chunks for the Dask array.
    - iteration: iteration number of filtering.
    
    Returns:
    - filtered_image: 3D numpy array, the filtered image.
    """
    # Convert the input image to a Dask array with specified chunks
    image_da = da.from_array(image, chunks=chunk_size)

    print(image_da.shape)
    
    for i in range(iteration):
        print("iteration " + str(i))
        # Apply the custom filter using map_overlap
        tmp_da = image_da.map_overlap(custom_filter,
                                      depth=1,
                                      boundary='nearest')
        filtered_image = tmp_da.compute()
        print(filtered_image.shape)
        image_da = da.from_array(filtered_image, chunks=chunk_size)
    
    return filtered_image



In [4]:
def dilate_segments():
    client = Client(n_workers=14, threads_per_worker=1)
    input_file_path = "D:/E65mEmbryo/E65SegFinal.tiff"
    output_file_path = "D:/E65mEmbryo/E65SegFinalD2.tiff"
    iteration = 2
    
    # Load the TIFF file as a NumPy array
    with tifffile.TiffFile(input_file_path) as tif:
        image = tif.asarray()

    filtered_image = apply_custom_filter_dask(image, chunk_size=(100, 100, 100), iteration=iteration)

    tifffile.imsave(output_file_path, filtered_image, compression=("ZLIB", 6))    

def main():
    dilate_segments()


if __name__ == '__main__':
    main()

C:\Users\liuz11\Anaconda3\envs\cellpose\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62044 instead
  warnings.warn(
C:\Users\liuz11\Anaconda3\envs\cellpose\lib\contextlib.py:120: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.06s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


(648, 1152, 1152)
iteration 0


C:\Users\liuz11\Anaconda3\envs\cellpose\lib\site-packages\distributed\client.py:3108: UserWarning: Sending large graph of size 1.60 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


(648, 1152, 1152)
iteration 1


C:\Users\liuz11\Anaconda3\envs\cellpose\lib\site-packages\distributed\client.py:3108: UserWarning: Sending large graph of size 1.60 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


(648, 1152, 1152)


C:\Users\liuz11\AppData\Local\Temp\ipykernel_11912\1241259583.py:13: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(output_file_path, filtered_image, compression=("ZLIB", 6))
